# 2023 / 01 / 04

In [1]:
using IntervalArithmetic
using LaTeXStrings

In [2]:
include("./concatImportanceMethod/concatImportanceMethod.jl")
include("./concatMatricesMethod/concatMatricesMethod.jl")
include("./display/concatImportanceMethod.jl")
include("./display/concatMatricesMethod.jl")
include("./display/interval.jl")
include("./display/twofoldInterval.jl")
include("./twofoldIntervalPCM/twofoldIntervalPCM.jl")

intervalPCM2TwofoldIntervalPCM

In [3]:
function concatMatricesMethod(A₁::Matrix{Interval{T}}, A₂::Matrix{Interval{T}}) where {T <: Real}
    A = intervalPCM2TwofoldIntervalPCM(A₁, A₂)
    display(L"\text{修正前の行列}")
    display(L"A = %$(twofoldIntervalMatrixLaTeXString(A))")

    result = solveConcatMatricesMethodLP(A)
    display(L"\text{LPの最適解}")
    displayConcatMatricesMethodLPResult(result)

    Â = updateConcatMatricesMethodPCM(A, result)
    display(L"\text{修正後の行列}")
    display(L"\hat{A} = %$(twofoldIntervalMatrixLaTeXString(Â))")
end

concatMatricesMethod (generic function with 1 method)

In [4]:
function concatImportanceMethod(A₁::Matrix{Interval{T}}, A₂::Matrix{Interval{T}}) where {T <: Real}
    approximationResults = solveConcatImportanceMethodApproximationLP.([A₁, A₂])
    display(L"\text{各DMの上近似と下近似}")
    displayConcatImportanceMethodApproximationLPResults(approximationResults)

    tBoundaries = calculateConcatImportanceMethodTBoundaries.(approximationResults)
    display(L"\text{各DMの上近似と下近似の}t\text{の範囲}")
    displayConcatImportanceMethodTBoundaries(tBoundaries)

    result = solveConcatImportanceMethodConcatLP(approximationResults, tBoundaries)
    display(L"\text{LPの最適解}")
    displayConcatImportanceMethodConcatLPResult(result)

    Â = generateConcatImportanceMethodPCM(result)
    display(L"\text{結合後のPCM}")
    display(L"\hat{A} = %$(twofoldIntervalMatrixLaTeXString(Â))")
end

concatImportanceMethod (generic function with 1 method)

In [5]:
function runBothMethod(A₁::Matrix{Interval{T}}, A₂::Matrix{Interval{T}}) where {T <: Real}
    display(L"A_1 = %$(intervalMatrixLaTeXString(A₁))")
    display(L"A_2 = %$(intervalMatrixLaTeXString(A₂))")
    
    display(L"\text{--- PCMを結合 ---}")
    concatMatricesMethod(A₁, A₂)

    display(L"\text{--- 重要度を結合 ---}")
    concatImportanceMethod(A₁, A₂)
end

runBothMethod (generic function with 1 method)

結合前の順序が同じ場合に結合後も順序が保存されるかの検証

In [63]:
A₁ = [
    1..1 1.05..1.1 1.1..1.21
    (1/1.1)..(1/1.05) 1..1 1.1..1.22
    (1/1.21)..(1/1.1) (1/1.22)..(1/1.1) 1..1
]
A₂ = [
    1..1 2.8..3 1..2.8
    (1/3)..(1/2.8) 1..1 (1/8)..(1/3)
    (1/2.8)..1 3..8 1..1
]

display(L"A_1 = %$(intervalMatrixLaTeXString(A₁))")
display(L"A_2 = %$(intervalMatrixLaTeXString(A₂))")

concatImportanceMethod(A₁, A₂)

L"$A_1 = \begin{bmatrix} \left[ 1.0, 1.0 \right] & \left[ 1.05, 1.1 \right] & \left[ 1.1, 1.21 \right] \\ \left[ 0.909, 0.952 \right] & \left[ 1.0, 1.0 \right] & \left[ 1.1, 1.22 \right] \\ \left[ 0.826, 0.909 \right] & \left[ 0.82, 0.909 \right] & \left[ 1.0, 1.0 \right] \end{bmatrix}$"

L"$A_2 = \begin{bmatrix} \left[ 1.0, 1.0 \right] & \left[ 2.8, 3.0 \right] & \left[ 1.0, 2.8 \right] \\ \left[ 0.333, 0.357 \right] & \left[ 1.0, 1.0 \right] & \left[ 0.125, 0.333 \right] \\ \left[ 0.357, 1.0 \right] & \left[ 3.0, 8.0 \right] & \left[ 1.0, 1.0 \right] \end{bmatrix}$"

L"$\text{各DMの上近似と下近似}$"

L"$    w_{1}^{\text{L}-} = \begin{bmatrix} 0.355  \\  0.338  \\  0.307 \end{bmatrix}, ~~
    w_{1}^{\text{U}-} = \begin{bmatrix} 0.355  \\  0.338  \\  0.307 \end{bmatrix}, ~~
    w_{1}^{\text{L}+} = \begin{bmatrix} 0.355  \\  0.323  \\  0.291 \end{bmatrix}, ~~
    w_{1}^{\text{U}+} = \begin{bmatrix} 0.355  \\  0.355  \\  0.323 \end{bmatrix}
$"

L"$    w_{2}^{\text{L}-} = \begin{bmatrix} 0.429  \\  0.143  \\  0.429 \end{bmatrix}, ~~
    w_{2}^{\text{U}-} = \begin{bmatrix} 0.429  \\  0.143  \\  0.429 \end{bmatrix}, ~~
    w_{2}^{\text{L}+} = \begin{bmatrix} 0.429  \\  0.054  \\  0.246 \end{bmatrix}, ~~
    w_{2}^{\text{U}+} = \begin{bmatrix} 0.69  \\  0.153  \\  0.429 \end{bmatrix}
$"

...

L"$W = \begin{bmatrix}\left[ 0.424, 0.429 \right] \\ \left[ 0.147, 0.264 \right] \\ \left[ 0.307, 0.429 \right]\end{bmatrix}$"

L"$v^{\text{L}-} = \begin{bmatrix} 0.355  \\  0.143  \\  0.307 \end{bmatrix}, ~~
v^{\text{U}-} = \begin{bmatrix} 0.429  \\  0.338  \\  0.429 \end{bmatrix}, ~~
v^{\text{L}+} = \begin{bmatrix} 0.424  \\  0.323  \\  0.291 \end{bmatrix}, ~~
v^{\text{U}+} = \begin{bmatrix} 0.424  \\  0.323  \\  0.291 \end{bmatrix}
$"

L"$\varepsilon^\text{L} = \begin{bmatrix} 0.069  \\  0.18  \\  0.0 \end{bmatrix}, ~~
\varepsilon^\text{U} = \begin{bmatrix} 0.074  \\  0.186  \\  0.106 \end{bmatrix}
$"

L"$\text{結合後のPCM}$"

L"$\hat{A} = \begin{bmatrix} \left[   1.0, \left[   1.0,   1.0 \right],   1.0 \right] & \left[ 0.833, \emptyset, 3.383 \right] & \left[ 0.828, \emptyset, 1.621 \right] \\ \left[ 0.296, \emptyset,   1.2 \right] & \left[   1.0, \left[   1.0,   1.0 \right],   1.0 \right] & \left[ 0.333, \emptyset, 1.657 \right] \\ \left[ 0.617, \emptyset, 1.208 \right] & \left[ 0.604, \emptyset,   3.0 \right] & \left[   1.0, \left[   1.0,   1.0 \right],   1.0 \right] \end{bmatrix}$"

In [69]:
A₁ = [
    1..1 1..1 1..1
    1..1 1..1 1..4
    1..1 (1/4)..1 1..1
]
A₂ = [
    1..1 (1/3)..2 1..2
    (1/2)..3 1..1 1..1
    (1/2)..1 1..1 1..1
]

display(L"A_1 = %$(intervalMatrixLaTeXString(A₁))")
display(L"A_2 = %$(intervalMatrixLaTeXString(A₂))")

concatImportanceMethod(A₁, A₂)

L"$A_1 = \begin{bmatrix} \left[ 1.0, 1.0 \right] & \left[ 1.0, 1.0 \right] & \left[ 1.0, 1.0 \right] \\ \left[ 1.0, 1.0 \right] & \left[ 1.0, 1.0 \right] & \left[ 1.0, 4.0 \right] \\ \left[ 1.0, 1.0 \right] & \left[ 0.25, 1.0 \right] & \left[ 1.0, 1.0 \right] \end{bmatrix}$"

L"$A_2 = \begin{bmatrix} \left[ 1.0, 1.0 \right] & \left[ 1.0, 2.0 \right] & \left[ 1.0, 2.0 \right] \\ \left[ 0.5, 1.0 \right] & \left[ 1.0, 1.0 \right] & \left[ 1.0, 1.0 \right] \\ \left[ 0.5, 1.0 \right] & \left[ 1.0, 1.0 \right] & \left[ 1.0, 1.0 \right] \end{bmatrix}$"

L"$\text{各DMの上近似と下近似}$"

L"$    w_{1}^{\text{L}-} = \begin{bmatrix} 0.333  \\  0.333  \\  0.333 \end{bmatrix}, ~~
    w_{1}^{\text{U}-} = \begin{bmatrix} 0.333  \\  0.333  \\  0.333 \end{bmatrix}, ~~
    w_{1}^{\text{L}+} = \begin{bmatrix} 0.333  \\  0.333  \\  0.133 \end{bmatrix}, ~~
    w_{1}^{\text{U}+} = \begin{bmatrix} 0.333  \\  0.533  \\  0.333 \end{bmatrix}
$"

L"$    w_{2}^{\text{L}-} = \begin{bmatrix} 0.5  \\  0.25  \\  0.25 \end{bmatrix}, ~~
    w_{2}^{\text{U}-} = \begin{bmatrix} 0.5  \\  0.25  \\  0.25 \end{bmatrix}, ~~
    w_{2}^{\text{L}+} = \begin{bmatrix} 0.333  \\  0.25  \\  0.25 \end{bmatrix}, ~~
    w_{2}^{\text{U}+} = \begin{bmatrix} 0.5  \\  0.333  \\  0.333 \end{bmatrix}
$"

...

L"$W = \begin{bmatrix}\left[ 0.333, 0.417 \right] \\ \left[ 0.333, 0.333 \right] \\ \left[ 0.25, 0.333 \right]\end{bmatrix}$"

L"$v^{\text{L}-} = \begin{bmatrix} 0.333  \\  0.25  \\  0.25 \end{bmatrix}, ~~
v^{\text{U}-} = \begin{bmatrix} 0.5  \\  0.333  \\  0.333 \end{bmatrix}, ~~
v^{\text{L}+} = \begin{bmatrix} 0.333  \\  0.333  \\  0.25 \end{bmatrix}, ~~
v^{\text{U}+} = \begin{bmatrix} 0.333  \\  0.333  \\  0.25 \end{bmatrix}
$"

L"$\varepsilon^\text{L} = \begin{bmatrix} 0.0  \\  0.083  \\  0.0 \end{bmatrix}, ~~
\varepsilon^\text{U} = \begin{bmatrix} 0.167  \\  0.0  \\  0.0 \end{bmatrix}
$"

L"$\text{結合後のPCM}$"

"\$\\hat{A} = \\begin{bmatrix} \\left[   1.0, \\left[   1.0,   1.0 \\right],   1.0 \\right] & \\left[   1.0, \\left[   1.0,   1.0 \\right], 1.667 \\right] & \\left[   1.0, \\left[   1.0, 1.333 \\right],   2.0 \\right] \\\\ \\left[   0.6, \\left[   1.0,   1.0 \\right],   1.0 \\right] & \\left[ " ⋯ 30 bytes ⋯ "ght],   1.0 \\right] & \\left[  0.75, \\left[   1.0, 1.333 \\right], 1.333 \\right] \\\\ \\left[   0.5, \\left[  0.75,   1.0 \\right],   1.0 \\right] & \\left[  0.75, \\left[  0.75,   1.0 \\right], 1.333 \\right] & \\left[   1.0, \\left[   1.0,   1.0 \\right],   1.0 \\right] \\end{bmatrix}\$"